In [1]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from sympy import sympify
import sys; sys.path.insert(0, "../"); from utils import *
from models import SympyTorch, PartialDerivativeCalculator, CoeffLearner
from parametric_discovery_pinn import ParametricPINN, BurgerPINN, FinalParametricPINN
from pytorch_robust_pca import *
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR
from parametric_pde_diff import TrainSGTRidge, create_groups

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib
Running Python 3.9.6
You can use npar for np.array


In [2]:
data = pickle_load("../parametric_pde_data/parametric_burgers.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]

Exact = data['u']
X, T = np.meshgrid(x, t)

# Adding noise
noise_intensity = 0.01
noisy_xt = True

Exact = perturb(Exact, intensity=noise_intensity, noise_type="normal")
print("Perturbed Exact with intensity =", float(noise_intensity))

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# Add noise to (x, t) before setting the lb, and ub.
if noisy_xt: 
    print("Noisy (x, t)")
    X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# Robust PCA
print("Running Robust PCA...")
rpca = R_pca_numpy(X_train)
X_train_L, X_train_S = rpca.fit(tol=1e-16, max_iter=20000, iter_print=100)
print('Robust PCA Loss:', mean_squared_error(X_train, X_train_L+X_train_S))
X_train_S = (X_train_S-X_train_S.min())-(X_train_S.max()-X_train_S.min())
X_train_L = to_tensor(X_train_L, True)
X_train_S = to_tensor(X_train_S, True)

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

u_xx_true = 0.1*np.ones(time_dims)
uu_x_true = -1*(1+0.25*np.sin(t))

feature_names = ['u', 'u_x', 'u_xx']

# del X_train

Loaded from ../parametric_pde_data/parametric_burgers.pkl
Perturbed Exact with intensity = 0.01
Noisy (x, t)
Running Robust PCA...
iteration: 1, error: 11.025163838699909
iteration: 100, error: 0.01586064477732947
iteration: 200, error: 0.013130648013694625
iteration: 300, error: 0.01110095246201438
iteration: 400, error: 0.009638206649671702
iteration: 500, error: 0.008338545419338744
iteration: 600, error: 0.007103386667828245
iteration: 700, error: 0.006119223858490207
iteration: 800, error: 0.005411263089841483
iteration: 900, error: 0.004759830785728398
iteration: 1000, error: 0.004285066990731387
iteration: 1100, error: 0.0036898460954346225
iteration: 1200, error: 0.0030984335793701676
iteration: 1300, error: 0.002861052807901675
iteration: 1400, error: 0.0025924419460598516
iteration: 1500, error: 0.002327762476986574
iteration: 1600, error: 0.0020201431173777614
iteration: 1700, error: 0.0016671727386530905
iteration: 1800, error: 0.0011552027798738304
iteration: 1900, error: 

In [3]:
pinn = ParametricPINN(scale=False, lb=lb, ub=ub)
print("Loaded the pretrained weights")
pinn.load_state_dict(torch.load("./saved_path_inverse_parametric_burgers/parametric_pinn.pth"))
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)

pde_terms = ["u*u_x", "u_xx"]
func_terms = ["-0.1872898*sin(t)-1.0238724", "0.09875935"]
final_burger_pinn = FinalParametricPINN(model=model, pde_terms=pde_terms, func_terms=func_terms, uncert=True, scale=pinn.scale, lb=pinn.lb, ub=pinn.ub)
del pinn

Loaded the pretrained weights


In [4]:
class RobustFinalParametricPINN(nn.Module):
    def __init__(self, pinn, beta1=0.0, beta2=0.0, is_beta1_trainable=True, is_beta2_trainable=True, hidden_nodes=50):
        super(RobustFinalParametricPINN, self).__init__()
        self.pinn = pinn
        
        self.beta1 = beta1
        self.beta2 = beta2
        
        if is_beta1_trainable: self.beta1 = nn.Parameter(data=torch.tensor([self.beta1]), requires_grad=True)
        if is_beta2_trainable: self.beta2 = nn.Parameter(data=torch.tensor([self.beta2]), requires_grad=True)
        
        self.proj = nn.Sequential(nn.Linear(2, hidden_nodes), nn.Tanh(), nn.Linear(hidden_nodes, 2), nn.Tanh())
        
    def forward(self, x, t):
        return self.pinn(x, t)
            
    def loss(self, L, S, y_train):
        recov = L - self.beta2*self.proj(S)
        return self.pinn.loss(recov[:, 0:1], recov[:, 1:2], y_train)

In [5]:
robust_buger_pinn = RobustFinalParametricPINN(pinn=final_burger_pinn, is_beta1_trainable=False, hidden_nodes=32)

In [6]:
def pcgrad_closure(return_list=False):
    global N, X_train, X_train_L, X_train_S, u_train
    losses = robust_buger_pinn.loss(X_train, X_train_S, u_train)
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in robust_buger_pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(robust_buger_pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return losses[0]+losses[1]
    else: return losses

In [7]:
def finetuning_closure():
    global N, X_train, X_train_L, X_train_S, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    mse_loss, pde_loss = robust_buger_pinn.loss(X_train, X_train_S, u_train)
    loss = mse_loss + pde_loss
    if loss.requires_grad: loss.backward(retain_graph=True)
    return loss

In [8]:
# optimizer = MADGRAD(robust_buger_pinn.parameters(), lr=1e-6, momentum=0.9)
# robust_buger_pinn.train()
# for i in range(150):
#     optimizer.step(pcgrad_closure)
#     if i%10==0: print(pcgrad_closure(return_list=True))
        
f_opt = torch.optim.LBFGS(robust_buger_pinn.parameters(), lr=1e-1, max_iter=500, max_eval=int(1.25*500), history_size=300, line_search_fn='strong_wolfe')
final_burger_pinn.is_uncert = False
for i in range(50):
    f_opt.step(finetuning_closure)
    if i%10==0: print(finetuning_closure())

tensor(6.5629e-05, grad_fn=<AddBackward0>)
tensor(6.4337e-05, grad_fn=<AddBackward0>)
tensor(6.4337e-05, grad_fn=<AddBackward0>)
tensor(6.4337e-05, grad_fn=<AddBackward0>)
tensor(6.4337e-05, grad_fn=<AddBackward0>)


In [9]:
[x.item() for x in final_burger_pinn.pdc.parameters()]

[-0.9976454377174377, -0.24656158685684204, 0.10040841996669769]

In [10]:
errs = np.array([100*(1-0.9976454377174377), 100*(0.25-0.24656158685684204)/0.25, 100*(0.10040841996669769-0.1)/0.1])
errs.mean(), errs.std()

(0.6730804840723655, 0.5015855132780921)

In [12]:
robust_buger_pinn.beta2

Parameter containing:
tensor([-0.0503], requires_grad=True)

In [13]:
# save(robust_buger_pinn, "./saved_path_inverse_parametric_burgers/noisy2_final_rpca_parametric_pinn.pth")

In [14]:
# New results
# Noisy Exact & Noisy (x, t) & X_train = X_train - self.beta2*self.proj(S) | beta1 = 0.0
# params: [-1.0046403408050537, -0.24455536901950836, 0.1004636213183403]
# errs: (1.035169263680774, 0.8079990064924926)

In [15]:
# Clean Exact and clean (x, t) | final_parametric_pinn.pth
# params: [-1.0013394355773926, -0.2480146884918213, 0.0997583195567131]
# errs: (0.38991620143254785, 0.28918291006992664)

# Noisy Exact and clean (x, t)
# params: [-1.0029208660125732, -0.24306637048721313, 0.09983817487955093]
# errs: (1.075787842273714, 1.2016070382040953)

# Noisy Exact and Noisy (x, t)
# params: [-1.0028020143508911, -0.24078042805194855, 0.10122136771678925]
# errs: (1.729799310366311, 1.4368618683618857)

# Noisy Exact & Clean (x, t) & X_star = X_star_L+X_star_S
# params: [-1.000922441482544, -0.24657735228538513, 0.09988813102245331]
# errs: (0.5243907372156797, 0.5973244500071551)
# Noisy Exact & Noisy (x, t) & X_star = X_star_L+X_star_S
# params: [-1.0046403408050537, -0.24455536901950836, 0.1004636213183403]
# errs: (1.035169263680774, 0.8079990064924926)

In [16]:
# save(final_burger_pinn, "./saved_path_inverse_parametric_burgers/noisy_final_parametric_pinn.pth")
# save(final_burger_pinn, "./saved_path_inverse_parametric_burgers/noisy2_final_parametric_pinn.pth")